In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
import json

In [2]:
ct = contrans()

In [3]:
goodbioguide = ct.get_bioguide(name='good', state='Virginia', district=5)

In [4]:
goodid = goodbioguide.reset_index()['bioguideId'][0]

In [5]:
goodid

'G000595'

In [ ]:


        #def get_sponsored_legislation(self):
            #params = {'api_key':self.congresskey,
                      'limit':1}
            #headers = self.make_headers()
            #root = 'https://api.congress.gov/v3'
            #endpoint = 'member/{bioguideId}/sponsored-legislation'
            #r = requests.get(root + endpoint,
                             params=params,
                             headers=headers)
            

In [10]:
# Sponsored Legislation

root = 'https://api.congress.gov/v3'
endpoint = f'/member/{goodid}/sponsored-legislation'
headers = ct.make_headers()
params = {'api_key': congresskey,
        'limit': 1}
r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)     
r


NameError: name 'congresskey' is not defined

In [ ]:
r.text

In [ ]:
json.loads(r.text)['sponsoredLegislations'][0]

In [ ]:
bioguides[['name', 'state', 'district', 'bioguideId']]

In [ ]:
bioguides = ct.get_bioguideIDs()
bioguides

In [ ]:
members_df = ct.get_bioguideIDs()

In [ ]:
members_df

In [ ]:
votes = ct.get_votes()
votes

In [ ]:
house = votes.query('chamber == "House"')
house = house[['rollnumber','icpsr','cast_code']]
house

In [ ]:
house_mat = pd.merge(house, house, on='rollnumber')
house_mat

In [ ]:

house_mat['agree'] = house_mat['cast_code_x'] == house_mat['cast_code_y']
house_mat = house_mat.groupby(['icpsr_x', 'icpsr_y']).agg({'agree':'mean'})
house_mat = house_mat.reset_index()
house_mat

In [ ]:
ideo = ct.get_ideology()
members = ideo[['icpsr', 'party_code', 'bioname']]
members